In [ ]:
import os
import time
import praw
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from bert_score import score as bert_score
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

# --- Load .env variables ---
load_dotenv()

# --- Reddit Authentication ---
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    username=os.getenv("REDDIT_USERNAME"),
    password=os.getenv("REDDIT_PASSWORD"),
    user_agent=os.getenv("REDDIT_USER_AGENT")
)

# --- Subreddit Mapping ---
SUBREDDITS = {
    0.2: "ai_debate_temp_02",
    0.8: "ai_debate_temp_04"
}

# --- LLM Factory ---
def get_llm(temp):
    return ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4", temperature=temp)

# --- Agent Definitions ---
def agent_replace_jobs(llm, msg):
    return llm([
        SystemMessage(content="You argue that AI will replace most human jobs."),
        HumanMessage(content=msg)
    ]).content

def agent_advance_dev(llm, msg):
    return llm([
        SystemMessage(content="You argue that AI will accelerate human development, not replace jobs."),
        HumanMessage(content=msg)
    ]).content

# --- Debate Simulation ---
def simulate_and_post_debate(temp, subreddit_name, turns=6):
    llm = get_llm(temp)
    transcript = []

    title = f"AI Debate at Temperature {temp} – Jobs vs Human Development"
    body = f"This is a {turns}-turn AI-generated debate between two agents.\n\nTemperature = {temp}"
    subreddit = reddit.subreddit(subreddit_name)
    submission = subreddit.submit(title=title, selftext=body)
    print(f"[Posted] r/{subreddit_name}: {title}")

    # Start with initial prompt
    initial_prompt = "AI is rapidly automating jobs in every field. Why shouldn't we fear mass unemployment?"
    first_reply = agent_replace_jobs(llm, initial_prompt)
    parent = submission.reply(f"**AI-Replaces-Jobs**: {first_reply}")
    transcript.append(("Replace", first_reply))
    time.sleep(10)

    # Alternate turns
    msg = first_reply
    for i in range(1, turns):
        if i % 2 == 0:
            response = agent_replace_jobs(llm, msg)
            tag = "**AI-Replaces-Jobs**"
        else:
            response = agent_advance_dev(llm, msg)
            tag = "**AI-Advances-Dev**"

        parent = parent.reply(f"{tag}: {response}")
        transcript.append((tag, response))
        msg = response
        time.sleep(10)

    # Evaluate with BERTScore
    refs = [t[1] for t in transcript[:-1]]
    hyps = [t[1] for t in transcript[1:]]
    _, _, F1 = bert_score(hyps, refs, lang="en", verbose=False)
    avg_score = float(F1.mean())
    print(f"[Done] Debate at temp {temp} | BERTScore F1: {avg_score:.4f}")
    return avg_score

# --- Main Entry ---
if __name__ == "__main__":
    temperatures = [0.2, 0.4]
    scores = []

    for temp in temperatures:
        score = simulate_and_post_debate(temp, SUBREDDITS[temp])
        scores.append(score)

    # Plot Results
    plt.figure(figsize=(8, 5))
    plt.plot(temperatures, scores, marker='o')
    plt.title("BERTScore F1 vs Temperature (Reddit Debate Threads)")
    plt.xlabel("Temperature")
    plt.ylabel("Avg BERTScore F1")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


[Posted] r/ai_debate_temp_02: AI Debate at Temperature 0.2 – Jobs vs Human Development


C:\Users\donth\AppData\Local\Temp\ipykernel_123148\1502840595.py:34: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm([


NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}